In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup, Comment
import json
import re
from tqdm import tqdm_notebook as tqdm
import csv

In [2]:
def parse_info_from_page(soup, page_id):
    author = soup.find_all('div', class_='author')[0].text[3:]
    title = soup.find('div', class_='title').text
    substance = soup.find('div', class_='substance').text
    try:
        weight = soup.find('td', class_='bodyweight-amount').text
    except:
        weight = ''

    # Dosing Chart
    times = [x.text.split('T+ ')[-1] for x in soup.find_all('td', align="right")][:-1]
    amounts = [x.text for x in soup.find_all('td', class_='dosechart-amount')]
    methods = [x.text for x in soup.find_all('td', class_='dosechart-method')]
    substances = [x.text for x in soup.find_all('td', class_='dosechart-substance')]
    form = [x.text for x in soup.findAll('td', class_='dosechart-form')]

    keys = ['times', 'amounts', 'methods', 'substances', 'form']
    values = [times, amounts, methods, substances, form]

    dose_chart = dict(zip(keys, values))

    # Get Text
    comments = soup.findAll(text=lambda text:isinstance(text, Comment))
    start = comments.index(' Start Body ')
    gen = comments[start].nextGenerator()

    text = []
    for text_chunk in gen:
        if text_chunk != ' End Body ':
            text.append(str(text_chunk))
        else:
            break
    text = ''.join(text).replace('<br/>', '').replace('<br>', '')
    footer = soup.find_all('table', class_='footdata')[0].text.split('\n')
    exp_year = footer[1][10:14]
    gender = re.findall('(Female|Male)', footer[2])
    age = re.findall('[0-9]+', footer[3])
    views = footer[4].split('Views: ')[-1].replace(',','')
    date_published = footer[4].split('Views: ')[0].split('Published: ')[-1]

    parsed_info = [page_id, author, title, text, substance, weight, age, gender, dose_chart, exp_year, date_published, views]
    
    return parsed_info

In [4]:
base_url = 'https://www.erowid.org/experiences/exp.php?ID='
page_id = 6179
page = requests.get(base_url+str(page_id))
soup = BeautifulSoup(page.content, "html.parser")
soup
info = parse_info_from_page(soup, page_id)

<html>
<head>
<title>5-MeO-DMT - Erowid Exp - 'From Pleasantries to Regrets'</title>
<meta content="An Experience with 5-MeO-DMT. 'From Pleasantries to Regrets' by Birdseye" name="description"/>
<meta content="Experience Report Vaults drug trip reports stories descriptions" name="keywords"/>
<link href="/includes/general_default.css" rel="stylesheet" type="text/css"/>
<link href="includes/exp_view.css" id="main_css" rel="stylesheet" type="text/css"/>
<link href="includes/exp_view_light_on_dark.css" id="color_css" rel="stylesheet" type="text/css"/>
<script src="/includes/javascript/e_cookie_lib.js" type="text/javascript"></script>
<script src="/experiences/includes/exp_view.js" type="text/javascript"></script>
</head>
<body onload="expSetColors();">
<table align="center" border="0" cellpadding="0" cellspacing="0"><tr><td>
<img border="0" height="75" src="/experiences/images/exp_header_blk.gif" usemap="#exp_header" width="707"/>
</td></tr><tr height="3"><td></td></tr>
<tr><td align="cent

In [3]:
base_url = 'https://www.erowid.org/experiences/exp.php?ID='

with open("experiences.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(['page_id', 'author', 'title', 'text', 'substance', 'weight', 'age', 'gender', 'dose_chart', 'exp_year', 'date_published', 'views'])
    for page_id in (range(30000,34500)):
        print(page_id)
        page = requests.get(base_url+str(page_id))
        soup = BeautifulSoup(page.content, "html.parser")
        try:
            info = parse_info_from_page(soup, page_id)
            writer.writerow(info)
        except:
            pass

30000


SSLError: HTTPSConnectionPool(host='www.erowid.org', port=443): Max retries exceeded with url: /experiences/exp.php?ID=30000 (Caused by SSLError(SSLCertVerificationError("hostname 'www.erowid.org' doesn't match 'search.dnsadvantage.com'")))

In [7]:
info

[14,
 'bendyguy',
 'Life-Line',
 '\n\n<span class="erowid-warning">[Erowid Note: \nMost Coricidin contains CPM (Chlorpheniramine Maleate) which can be dangerous in high doses. See <a href="/chemicals/dxm/dxm_info2.shtml">DXM Brand Warnings</a> for more info.]</span>[Erowid Note: \nMost Coricidin contains CPM (Chlorpheniramine Maleate) which can be dangerous in high doses. See <a href="/chemicals/dxm/dxm_info2.shtml">DXM Brand Warnings</a>DXM Brand Warnings for more info.] \n\nMy friend \'Bill\' and I were going to do dxm together for the first time.  He was experienced, but more with the robitussin side of the field.  We would be trying Coricidin cough tablets, 30mg per pill.  We bought three boxes and split them up evenly.  Using gatorade to wash them down, the plan was to take them one at a time every minute until we were done, starting at 7 pm.\r\n\nWe would be watching \'What dreams may come\' for the stunningly vivid colors and grand theme until we started to feel some heavier eff